In [9]:
## In-Class Assignment 15 - Evan Callaghan

import boto3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import recall_score

## Defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'data-445-bucket-callaghan'
bucket = s3.Bucket(bucket_name)

## Defining the csv file
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## 1. Using the pandas library to read the csv data file and create a data-frame called heart
heart = pd.read_csv(file_content_stream)

heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [10]:
## 2. Removing observations with missing values

heart = heart.dropna()

In [11]:
## 3. Using male, age, currentSmoker, totChol, sysBP,BMI, heartRate, and glucose as the predictor variables and TenYearCHD 
## as the target variable to split the data into two data-frames (taking into account the proportion of 0s and 1s): 
## train (80%) and test (20%)

X = heart[['male', 'age', 'currentSmoker', 'totChol', 'sysBP', 'BMI', 'heartRate', 'glucose']]
Y = heart['TenYearCHD']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [12]:
## 4. Using the train data-frame to build an adaboost model (with 500 trees and max tree depth equal 3) in which: 
## male, age, currentSmoker, totChol, sysBP,BMI, heartRate, and glucose are the predictor variables, and TenYearCHD is the target variable

## Building the model
md1 = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500).fit(X_train, Y_train)

## Extracting variable importance
md1.feature_importances_

## Top 5: age, totChol, sysBP, BMI, glucose

array([0.01609932, 0.11416639, 0.01292099, 0.18016261, 0.17987095,
       0.23969059, 0.10845223, 0.14863692])

In [14]:
## Using the top 5 variables from the adaboost model (the variables with the highest importance values) to build an adaboost
## model (with 500 trees and max tree depth equal 3) in which TenYearCHD is the target variable

X_train_ada = X_train[['age', 'totChol', 'sysBP', 'BMI', 'glucose']]
X_test_ada = X_test[['age', 'totChol', 'sysBP', 'BMI', 'glucose']]

## Building the model
md1_top5 = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500).fit(X_train_ada, Y_train)

## Predicting on the test set
md1_preds = md1_top5.predict_proba(X_test_ada)[:, 1]

## Using 10% as cutoff value and reporting the recall
md1_preds = np.where(md1_preds < 0.1, 0, 1)

print('Recall score of AdaBoost Model:', recall_score(Y_test, md1_preds))

Recall score of AdaBoost Model: 1.0


In [15]:
## 5. Using the train data-frame to build a random forest model (with 500 trees and max tree depth equal 3) in which: 
## male, age, currentSmoker, totChol, sysBP,BMI, heartRate, and glucose are the predictor variables, and TenYearCHD is the target variable

md2 = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)

md2.feature_importances_

## Top 5: age, sysBP, glucose, totChol, BMI

array([0.04415608, 0.31753208, 0.00524973, 0.08100804, 0.32213431,
       0.06744544, 0.03320186, 0.12927246])

In [16]:
## Using the top 5 variables from the random forest model (the variables with the highest importance values) to build a random forest
## model (with 500 trees and max tree depth equal 3) in which TenYearCHD is the target variable

X_train_rf = X_train[['age', 'sysBP', 'glucose', 'totChol', 'BMI']]
X_test_rf = X_test[['age', 'sysBP', 'glucose', 'totChol', 'BMI']]

## Building the model
md2_top5 = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train_rf, Y_train)

## Predicting on the test set
md2_preds = md2_top5.predict_proba(X_test_rf)[:, 1]

## Using 10% as cutoff value and reporting the recall.
md2_preds = np.where(md2_preds < 0.1, 0, 1)

print('Recall score of Random Forest Model:', recall_score(Y_test, md2_preds))

Recall score of Random Forest Model: 0.7410714285714286


In [ ]:
## 6. Using the results from parts (4) and (5), we would select the AdaBoost model to predict TenYearCHD
## because it has the higher recall score of the two considered models.